## Import libraries

In [1]:
import os
import gspread
from google.oauth2.service_account import Credentials

import pandas as pd
from pandas import DataFrame
from typing import Optional, Literal

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Load & Update Google sheet

In [3]:
scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = Credentials.from_service_account_file('../credentials.json', scopes=scopes)
client = gspread.authorize(creds)

In [4]:
sheet_id = os.environ['SHEET_ID']
sheet = client.open_by_key(sheet_id)

### Load worksheet

In [5]:
def load_worksheet(sheet_id: Optional[str] = None, sheet_idx: Optional[int] = None) -> DataFrame:
    if sheet_id is None and sheet_idx is None:
        raise Exception("Please provide either a sheet id or sheet index.")

    worksheet = None
    if sheet_id:
        worksheet = sheet.get_worksheet_by_id(sheet_id)
    else:
        worksheet = sheet.get_worksheet(sheet_idx)

    return pd.DataFrame(worksheet.get_all_records())

In [6]:
df = load_worksheet(sheet_idx=0)
df

,Col 1,Col 2
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


### Update worksheet

In [7]:
def update_worksheet(dataframe: DataFrame, sheet_id: Optional[str] = None, sheet_idx: Optional[int] = None, mode: Literal['update', 'replace'] = 'update') -> None:
    if sheet_id is None and sheet_idx is None:
        raise Exception("Please provide either `sheet_id` or `sheet_idx")
    
    # get worksheet
    if sheet_id:
        worksheet = sheet.get_worksheet_by_id(sheet_id)
    else:
        worksheet = sheet.get_worksheet(sheet_idx)

    # update worksheet
    if mode == 'replace':
        worksheet.clear()
    
    worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

In [8]:
new_df = pd.DataFrame({'col a': [1,2,3,4,5], 'col b': [11,12,13,14,15]})
update_worksheet(new_df, sheet_idx=0, mode='replace')

In [9]:
df = load_worksheet(sheet_idx=0)
df

,col a,col b
0,1,11
1,2,12
2,3,13
3,4,14
4,5,15
